In [2]:
#import packages
import numpy as np
import scanpy as sc
from collections import OrderedDict
import scipy
import pandas as pd

#spectra imports 
from spectra import spectra as spc
from spectra import spectra_util as util

/home/clinic/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
adata = sc.read_h5ad("../../clean_myeloid.h5ad")

In [9]:
adata.obs_keys()

['Clusters', 'Celltype', 'SampleID', 'name', 'background', 'condition', 'replicate', 'experiment', 'ncells', 'libsize', 'mito_fraction', 'ribo_fraction', 'mhc_fraction', 'actin_fraction', 'cytoskeleton_fraction', 'malat1_fraction', 'scrublet_predict', 'scrublet_score', 'lowlibsize', 'Clusters_myeloid', 'Celltype_myeloid']


In [1]:
annotations = {
    'global': {},
    'Alveolar Macrophage': {
        'gene-set':['MARCO', 'KRT79', 'KRT19', 'CAR4', 'CHIL3']
    },
    'ARG1 Macrophage': {
        'mouse-1':['CCL17', 'CCR5', 'MAFB', 'CD86', 'ARG1'],
        'mouse-2':['GPNMB', 'SPP1', 'CD68']
    },
    'C1QA Macrophage': {
        'canonical':['CSF1R', 'MRC1', 'APOE', 'C1QA', 'C1QB', 'C1QC', 'CTSB', 'CTSD']
    },
    'CCR7 cDC2': {
        'gene-set':['FSCN1', 'CCR7', 'LY75', 'CCL22', 'CD40', 'BIRC3', 'NFKB2', 'IL12B',
            'MARCKSL1', 'CD274', 'TNFRSF9', 'SEMA7A', 'STAT4']
    },
    'cDC1': {
        'gene-set':['XCR1', 'CLEC9A', 'CADM1', 'BATF3', 'IRF8']
    },
    'cDC2': {
        'gene-set':['ZBTB46', 'CD207', 'IRF4', 'CEBPB', 'LILRB4A', 'ITGAX']
    },
    'CSF3R Monocyte': {
        'neutrophil-like':['S100A8', 'S100A9', 'CSF3R', 'IL1B']
    },
    'Monocyte': {
        'classical':['FN1', 'F13A1', 'VCAN', 'LY6C1', 'LY6C2', 'CCR2'],
        'non-classical':['ITGAL', 'CX3CR1', 'FCGR4', 'CD300E', 'ACE']
    },
    'pDC': {
        'gene-set':['TCF4', 'PLD4', 'BCL11A', 'RUNX2', 'SIGLECH', 'CCR9', 'BST2']
    }
}

In [5]:
for cell_type_dict in annotations.values():
    for gene_set in cell_type_dict.values():
        for gene in gene_set:
            if gene not in adata.var.index:
                print(gene)

In [6]:
vocab = adata.var_names
gene2id = dict((v, idx) for idx, v in enumerate(vocab))
word2id = dict((v, idx) for idx, v in enumerate(vocab))

In [11]:
def check_gene_set_dictionary(adata, annotations, obs_key='cell_type_annotations',global_key='global', return_dict = True):
    '''
    Filters annotations dictionary contains only genes contained in the adata. 
    Checks that annotations dictionary cell type keys and adata cell types are identical.
    Checks that all gene sets in annotations dictionary contain >2 genes after filtering.
    
    adata: AnnData , data to use with Spectra
    annotations: dict , gene set annotations dictionary to use with Spectra
    obs_key: str , column name for cell type annotations in adata.obs
    global_key: str , key for global gene sests in gene set annotation dictionary
    return_dict: bool , return filtered gene set annotation dictionary
    
    returns: dict , filtered gene set annotation dictionary
    
    '''
    #test if keys match
    adata_labels  = list(set(adata.obs[obs_key]))+['global']#cell type labels in adata object
    annotation_labels = list(annotations.keys())
    matching_celltype_labels = list(set(adata_labels).intersection(annotation_labels))
    if set(annotation_labels)==set(adata_labels):
        print('Cell type labels in gene set annotation dictionary and AnnData object are identical')
        dict_keys_OK = True
    if len(annotation_labels):
        Counter = 0
        annotations_new = {}
        for k,v in annotations.items():
            annotations_new[k] = {}
            for k2,v2 in v.items():
                annotations_new[k][k2]= [x for x in v2 if x in adata.var_names]
                length = len(v2)
                if length<3:
                    print('gene set',k2,'for cell type',k,'is of length',length)
                    Counter = Counter+1
            
    if Counter > 0:
        print(Counter,'gene sets are too small. Gene sets must contain at least 3 genes')
    elif Counter == 0 and dict_keys_OK:
        print('Your gene set annotation dictionary is correctly formatted.')
    if return_dict:
        return annotations_new
     
check_gene_set_dictionary(adata, annotations, "Celltype_myeloid")

Cell type labels in gene set annotation dictionary and AnnData object are identical
Your gene set annotation dictionary is correctly formatted.


{'global': {},
 'Alveolar Macrophage': {'gene-set1': ['MARCO',
   'KRT79',
   'KRT19',
   'CAR4',
   'CHIL3']},
 'ARG1 Macrophage': {'mouse-1': ['CCL17', 'CCR5', 'MAFB', 'CD86', 'ARG1'],
  'mouse-2': ['GPNMB', 'SPP1', 'CD68']},
 'C1QA Macrophage': {'canonical': ['CSF1R',
   'MRC1',
   'APOE',
   'C1QA',
   'C1QB',
   'C1QC',
   'CTSB',
   'CTSD']},
 'CCR7 cDC2': {'gene-set2': ['FSCN1',
   'CCR7',
   'LY75',
   'CCL22',
   'CD40',
   'BIRC3',
   'NFKB2',
   'IL12B',
   'MARCKSL1',
   'CD274',
   'TNFRSF9',
   'SEMA7A',
   'STAT4']},
 'cDC1': {'gene-set3': ['XCR1', 'CLEC9A', 'CADM1', 'BATF3', 'IRF8']},
 'cDC2': {'gene-set4': ['ZBTB46',
   'CD207',
   'IRF4',
   'CEBPB',
   'LILRB4A',
   'ITGAX']},
 'CSF3R Monocyte': {'neutrophil-like': ['S100A8', 'S100A9', 'CSF3R', 'IL1B']},
 'Monocyte': {'classical': ['FN1', 'F13A1', 'VCAN', 'LY6C1', 'LY6C2', 'CCR2'],
  'non-classical': ['ITGAL', 'CX3CR1', 'FCGR4', 'CD300E', 'ACE']},
 'pDC': {'gene-set5': ['TCF4',
   'PLD4',
   'BCL11A',
   'RUNX2',
   

In [12]:
np.seterr('raise')

model = spc.est_spectra(adata = adata, gene_set_dictionary = annotations, 
                        use_highly_variable = False, cell_type_key = 'Celltype_myeloid',
                        use_weights = True, lam = 0.1, 
                        delta = 0.001,kappa = 0.00001, rho = 0.00001, 
                        use_cell_types = True, #set to False to not use the cell type annotations
                        n_top_vals = 25, 
                        num_epochs = 5000 #for demonstration purposes we will only run 2 epochs, we recommend 10,000 epochs
                       )

/home/clinic/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


FloatingPointError: invalid value encountered in double_scalars